In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [15]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import SemanticSimilarityExampleSelector

In [9]:
loader = DirectoryLoader('data/examples/', glob="**/*.txt", loader_cls=TextLoader)
docs = loader.load()

In [12]:
### Splitting the document into chunks and creating splits
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,add_start_index=True)
splits = text_splitter.split_documents(docs)

In [13]:
## Creating Open AI Embeddings
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])

In [14]:
## Loading the Vector data store
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [16]:
## Creating an example selector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

In [18]:
# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "Lambda_Python"})

[{'source': 'data\\examples\\lambdaFunction_s3tos3_copy.txt',
  'start_index': 0},
 {'source': 'data\\examples\\lambdaFunction_s3tos3_copy.txt',
  'start_index': 145}]

In [24]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

In [25]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [27]:
from langchain_community.chat_models import ChatAnthropic

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)
chain = final_prompt | llm

chain.invoke({"input": "Give an example of Lambda_Python"})

KeyError: 'input'